<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%206/Modulo_6_Ejercicio_6_Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejercicio 6: Predicción de Fallos con Modelos Híbridos (Stacking)**
**Propósito**: Construir un modelo de stacking combinando modelos tradicionales y avanzados para detectar fallos en procesos industriales.

## Introducción

La predicción de fallos en entornos industriales es un reto clave para garantizar la calidad y fiabilidad de los productos. En este ejercicio, abordamos este desafío aplicando una técnica avanzada conocida como **stacking** o **ensamblado jerárquico**.

El stacking permite **combinar varios modelos base** (de distinto tipo) y un **modelo meta** (meta-modelo) que aprende a partir de sus predicciones. El objetivo es construir un predictor más robusto, capaz de mejorar el rendimiento frente a modelos individuales.

Utilizaremos el **SECOM Manufacturing Data Set**, un conjunto de datos multivariable, complejo y realista, ideal para explorar técnicas de ensemble en problemas de clasificación binaria con ruido y correlaciones espurias.

Objetivos del ejercicio:
- Entrenar tres modelos base: **Regresión Logística**, **Random Forest**, **MLP**.
- Combinar sus predicciones utilizando un **meta-modelo** (XGBoost).
- Comparar el rendimiento del stacking frente a los modelos individuales.
- Discutir la aplicabilidad del stacking en entornos industriales.

> ### Recomendación:
> Las redes neuronales hacen millones de operaciones matemáticas (multiplicaciones de matrices, derivadas, etc.) durante el entrenamiento. Una CPU puede hacer estas operaciones, pero una GPU está diseñada específicamente para hacerlas en paralelo y mucho más rápido.
>
> Por ello te recomiendo que para estos ejercicios cambies tu entorno de ejecución en Colab.
>
> Puedes hacerlo haciendo click en `Entorno de Ejecución` en la barra superior, después `Cambiar tipo de entorno de ejecución` y selecciona `GPU`. Esto hará que tus notebook ejecuten más rápido los cálculos con Deep Learning.



**Licencia del dataset:** SECOM Manufacturing Data Set — Fuente: UCI ML Repository, Licencia: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)

## Carga de librerías y configuración del entorno

In [ ]:
# Celda 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier

from imblearn.over_sampling import SMOTE

import requests
import zipfile
import io

from xgboost import XGBClassifier

# Estilo visual
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

## Descripción del dataset o datos de entrada

El dataset utilizado en este ejercicio es el **SECOM Manufacturing Data Set**, proveniente del UCI Machine Learning Repository.

- Contiene mediciones de sensores durante procesos de manufactura electrónica.
- Cada fila representa una observación con cientos de variables numéricas.
- La variable objetivo es binaria (`1`: fallo, `-1`: sin fallo).

Este dataset se caracteriza por su **alta dimensionalidad**, **valores ruidosos** y **presencia de correlaciones espurias**, lo que lo convierte en un buen candidato para evaluar modelos de tipo ensemble como el stacking.

Fuente: https://archive.ics.uci.edu/ml/datasets/SECOM

## Desarrollo del código paso a paso

**Carga y exploración de los datos**

In [ ]:

zip_url = "https://github.com/dtoralg/IE_Calidad_ML/raw/refs/heads/main/Data/Modulo%206/secom.zip"

# Descargar el zip
response = requests.get(zip_url)

# Verificar que la descarga fue exitosa
if response.status_code == 200:
    # Leer el archivo ZIP en memoria
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))

    # Mostrar el contenido del ZIP
    print("Archivos contenidos:")
    print(zip_file.namelist())

    # Extraer todos los archivos (opcional)
    zip_file.extractall("secom_data")  # Carpeta donde se guardarán
    print("Archivos extraídos en la carpeta 'secom_data'.")
else:
    print("Error al descargar el archivo:", response.status_code)


In [ ]:
# Celda 2
url_data = '/content/secom_data/secom.data'
url_labels = '/content/secom_data/secom_labels.data'

# Cargar datos
df = pd.read_csv(url_data, sep='\s+', header=None)
labels = pd.read_csv(url_labels, sep='\s+', header=None)
df['target'] = labels[0]
df.head()

**Limpieza de datos y preparación del conjunto de entrenamiento**

In [ ]:
# Celda 3
# Eliminar columnas con demasiados nulos
... # eliminar columnas con más del 25% de NaNs

# Eliminar filas con valores nulos restantes
...

# Separar variables predictoras y target
X = ...
y = ...

# Escalar datos
...

# Separar entrenamiento y test
...

# Crear y aplicar SMOTE
...

# Verificar
print("Dimensiones finales:", X_train.shape, X_test.shape)


**Entrenamiento de modelos individuales (base learners)**

In [ ]:
# Celda 4
# Regresión Logística
...

# Random Forest
...

# MLP
...

## Visualización de resultados

**Evaluación de modelos individuales**

In [ ]:
# Celda 5
for name, model in [('Logistic Regression', lr), ('Random Forest', rf), ('MLP', mlp)]:
    y_pred = model.predict(X_test)
    print(f"--- {name} ---")
    print(classification_report(y_test, y_pred))
    print("F1 Macro:", f1_score(y_test, y_pred, average='macro'))
    print()

**Entrenamiento del modelo Stacking con meta-modelo XGBoost**

In [ ]:
# Celda 6
# Meta-modelo
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Definir ensemble stacking
stacking = StackingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('mlp', mlp)
    ],
    final_estimator=xgb,
    cv=5
)

# Entrenar ensemble
stacking.fit(X_train, y_train)

In [ ]:
# Celda 7
y_pred_stack = stacking.predict(X_test)
print("--- Modelo Stacking ---")
print(... ) # Classification Report
print("F1 Macro:", f1_score(... ))

## Conclusiones

...

## Próximos pasos

- Realizar selección de variables para reducir dimensionalidad antes de aplicar stacking.
- Implementar versiones avanzadas con validación interna y meta-predicciones.
- Probar meta-modelos distintos como LogisticRegression para evaluar su impacto.
